In [1]:
import numpy as np
import json
import re
import requests

import pandas as pd
import nltk
nltk.download('stopwords')
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing  import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.pipeline import Pipeline, FeatureUnion
from nltk.stem.snowball import RussianStemmer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score
from IPython.core.display import HTML, display

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maratgasanov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
TOKEN_PATTERN = "[а-яё]+"

In [3]:
train_df = pd.read_csv("train.csv")

train_df.head()

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [4]:
test_df = pd.read_csv("test.csv")

test_df.head()

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [5]:
train_df["target"].value_counts()

False    118594
True      16715
Name: target, dtype: int64

In [6]:
from sklearn.metrics import f1_score

In [7]:
X_train = train_df["title"].values
X_test = test_df["title"].values
y_train = train_df["target"].astype(int).values

In [8]:
stop_words = nltk.corpus.stopwords.words('russian')
stop_words

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [9]:
def preprocessor(text):
    whitespaced_text = re.sub("[^а-яё!:)(]", ' ', text.lower())
    return re.sub(' +', ' ',  whitespaced_text)

In [10]:
vectorizer = CountVectorizer(lowercase=True, ngram_range=(1, 1), stop_words=stop_words, token_pattern=TOKEN_PATTERN, min_df=5, max_df=0.8)
X_train_vec = vectorizer.fit_transform(X_train).toarray()
X_test_vec = vectorizer.fit_transform(X_test).toarray()

In [11]:
stemmer = RussianStemmer()
def preprocess_with_stem(text):
    return stemmer.stem(word=preprocessor(text))

In [12]:
tfidf_model = Pipeline([
    (
        'vectorizer',
        TfidfVectorizer(
            lowercase=True, ngram_range=(1, 1), analyzer='word',stop_words=stop_words,token_pattern=TOKEN_PATTERN,
            preprocessor=preprocessor, min_df=5, max_df=0.8
        )
    ),
    ('clf', MultinomialNB())
])

In [13]:
model = Pipeline([
    (
        'vectorizer',
        CountVectorizer(lowercase=True, ngram_range=(1, 1), stop_words=stop_words, token_pattern=TOKEN_PATTERN,
                        preprocessor=preprocessor)
    ),
    ('clf', MultinomialNB())
])

In [14]:
model.fit(X_train, y_train)
f1_score(y_train, model.predict(X_train))

0.6330751297285102

In [15]:
%%time
model = MultinomialNB()

model.fit(
    X_train_vec,
    y_train
)

y_pred = model.predict(
    X_train_vec
)

f1_score(y_train, y_pred)

CPU times: user 1min 46s, sys: 1min 9s, total: 2min 56s
Wall time: 3min 31s


0.626855725445189